### Exploration et propositions de visualisations pour l'onglet DATA

#### Contexte et objectifs


Objectif de l'onglet DATA : 
- Quels impacts sur les milieux naturels et quelles sont les comportements/usages/secteurs économiques à l'origine de cette pollution macrodéchets?​

Besoins des collectivités :​
-  Mieux connaître la problématique macrodéchets sur leur territoire​
-  Suivre l'efficacité des politiques publiques mises en œuvre​

Indicateurs déjà identifiés par Merterre : 
- Typologie des matériaux​
- Top déchets comptés,​
- Typologie des déchets (si niveau 4)​
- Secteurs économiques concernés, filières REP concernés, ​
- Filtres par milieu/lieu pur chacune de ces informations​
- Evolutions dans le temps

Analyses présentes dans le bilan pdf : 
- Typologie des matériaux 
    - Pie chart : part des matériaux en volume tous milieux confondus 
    - Stacked bar chart : part des matériaux en volume selon le type de milieu
    - Sorted bar chart : Principaux déchets comptés (en nombre d'objets)
- Typologie des déchets 
    - pie chart fragments vs objets identifiables
    - pie chart en nombre du matériau pour les fragments ramassés
    - part en nombre des objets identifiables ramassés
- Secteurs économiques : 
    - pie chart : part des déchets dont le secteur est identifiable
    - pie chart des secteurs économiques identifiés, tous milieux confondus
    - graph à barre des secteurs économiques identifiés en fonction du type de milieu

#### 0/ Import des données

In [14]:
#import librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

In [15]:
#Import data
PATH = "~/Documents/Projets data/D4G_12_zero_dechet_sauvage/Exploration_visualisation/data/"

# nombre maximum de colonnes à afficher = 1500
pd.set_option('display.max_columns', 1500)

# import données structures et niveaux de caracterisation et fichier geojson france
#df_niveaux_dechets = pd.read_excel(PATH+"Déchets-Niveaux2-3.xlsx")
#df_structures = pd.read_excel(PATH+"export_structures_29022024.xlsx")
gdf = gpd.read_file(PATH+"a-com2022.json")

# import du fichier excel des données nettoyées par le sript python "cleaning.py" dans un dataframe et visualisation des premieres lignes
df_enriched_bdd_zds = pd.read_csv(PATH+"data_zds_enriched.csv")
df_nbdechets = pd.read_csv(PATH+"data_releve_nb_dechet.csv")
display(df_enriched_bdd_zds.head())


,ID_RELEVE,LIEU_PAYS,LIEU_REGION,LIEU_DEPT,LIEU_VILLE,LIEU_CODE_POSTAL,LIEU_COORD_GPS,LIEU_COORD_GPS_X,LIEU_COORD_GPS_Y,NOM_ZONE,SPOT_A1S,SPOT_ID,TYPE_LIEU,TYPE_MILIEU,TYPE_LIEU2,TYPE_DECHET,LONGUEUR_LINEAIRE,SURFACE,ID_EVENEMENT,NOM_EVENEMENT,TYPE_EVENEMENT,EVENEMENT_ENVERGURE,DATE,ANNEE,NOM_STRUCTURE,ID_STRUCTURE,NB_PARTICIPANTS,DUREE,NIVEAU_CARAC,VERSION_PROTOCOLE,VOLUME_TOTAL,VOLUME_TOTAL_SACS,POIDS_TOTAL,VOLUME_VOLUMINEUX,POURCENTAGE_VOLUMINEUX,GLOBAL_POURCENTAGE_TOTAL,GLOBAL_POURCENTAGE_PLASTIQUE,GLOBAL_POURCENTAGE_CAOUTCHOUC,GLOBAL_POURCENTAGE_BOIS,GLOBAL_POURCENTAGE_TEXTILE,GLOBAL_POURCENTAGE_PAPIER,GLOBAL_POURCENTAGE_METAL,GLOBAL_POURCENTAGE_VERRE,GLOBAL_POURCENTAGE_AUTRE,GLOBAL_VOLUME_PLASTIQUE,GLOBAL_VOLUME_CAOUTCHOUC,GLOBAL_VOLUME_BOIS,GLOBAL_VOLUME_TEXTILE,GLOBAL_VOLUME_PAPIER,GLOBAL_VOLUME_METAL,GLOBAL_VOLUME_VERRE,GLOBAL_VOLUME_AUTRE,SACS_POURCENTAGE_PLASTIQUE,SACS_POURCENTAGE_CAOUTCHOUC,SACS_POURCENTAGE_BOIS,SACS_POURCENTAGE_TEXTILE,SACS_POURCENTAGE_PAPIER,SACS_POURCENTAGE_METAL,SACS_POURCENTAGE_VERRE,SACS_POURCENTAGE_AUTRE,SACS_VOLUME_PLASTIQUE,SACS_VOLUME_CAOUTCHOUC,SACS_VOLUME_BOIS,SACS_VOLUME_TEXTILE,SACS_VOLUME_PAPIER,SACS_VOLUME_METAL,SACS_VOLUME_VERRE,SACS_VOLUME_AUTRE,SACS_POIDS_PLASTIQUE,SACS_POIDS_CAOUTCHOUC,SACS_POIDS_BOIS,SACS_POIDS_TEXTILE,SACS_POIDS_PAPIER,SACS_POIDS_METAL,SACS_POIDS_VERRE,SACS_POIDS_AUTRE,VOLUMINEUX_POURCENTAGE_PLASTIQUE,VOLUMINEUX_POURCENTAGE_CAOUTCHOUC,VOLUMINEUX_POURCENTAGE_BOIS,VOLUMINEUX_POURCENTAGE_TEXTILE,VOLUMINEUX_POURCENTAGE_PAPIER,VOLUMINEUX_POURCENTAGE_METAL,VOLUMINEUX_POURCENTAGE_VERRE,VOLUMINEUX_POURCENTAGE_AUTRE,VOLUMINEUX_VOLUME_PLASTIQUE,VOLUMINEUX_VOLUME_CAOUTCHOUC,VOLUMINEUX_VOLUME_BOIS,VOLUMINEUX_VOLUME_TEXTILE,VOLUMINEUX_VOLUME_PAPIER,VOLUMINEUX_VOLUME_METAL,VOLUMINEUX_VOLUME_VERRE,VOLUMINEUX_VOLUME_AUTRE,COMMENTAIRE_VOLUMINEUX,AUTRES_DECHETS,NB_DECHETS_AUTRES,COMMENTAIRE,commune,INSEE_COM,DEP,REG,EPCI,NATURE_EPCI,BV2022,LIBEPCI,DEPARTEMENT,REGION,BASSIN_DE_VIE
0,12,France,Provence-Alpes-Côte d'Azur,13.0,Marseille,13008.0,"[""5.3416741728752"",""43.213786286934""]",5.341674,43.213786,Anse de la Maronaise,0,NaN,"Littoral (terrestre)/Plage (sable, galets, gra...",Littoral (terrestre),"Plage (sable, galets, gravillons)",Echoué,2500.0,5.855528e+04,178,Calanques Propres 2019 Maronaise,ramassage,Calanques Propres,2019-05-25,2019,MerTerre,145,100.0,5.0,2,1,503.0,458.0,57.0,45.0,8.946322,100,54.0,0.0,4.0,2.0,4.0,4.0,19.0,13.0,271.62,0.0,20.12,10.06,20.12,20.12,95.57,65.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,0.0,1.0,1.0,1.0,1.0,29.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Marseille,13055,13,93,200054807,ME,13055,Métropole d'Aix-Marseille-Provence,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,Marseille
1,14,France,Provence-Alpes-Côte d'Azur,13.0,Marseille,13008.0,"[""5.3429503182801"",""43.213941067785""]",5.342950,43.213941,Anse de la Maronaise,0,NaN,Littoral (terrestre)/Espace naturel en arrière...,Littoral (terrestre),Espace naturel en arrière plage/côte (calanque...,Présent au sol (abandonné),2770.0,1.522318e+05,178,Calanques Propres 2019 Maronaise,ramassage,Calanques Propres,2019-05-25,2019,MerTerre,145,93.0,5.0,2,1,1630.0,1627.0,49.0,3.0,0.184049,100,43.0,3.0,5.0,2.0,12.0,15.0,7.0,13.0,700.90,48.9,81.50,32.60,195.60,244.50,114.10,211.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,4.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,:;,0,NaN,Marseille,13055,13,93,200054807,ME,13055,Métropole d'Aix-Marseille-Provence,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,Marseille
2,16,France,Provence-Alpes-Côte d'Azur,13.0,Cassis,13260.0,"[""5.5392828683314"",""43.226082263728""]",5.539283,43.226082,Vallat dres Brayes et bords des routes des ale...,0,NaN,Zone naturelle ou rurale (hors littoral et mon...,Zone naturelle ou rurale (hors littoral et mon...,Multi-lieux,Présent au sol (abandonné),1500.0,4.347502e+04,222,Nettoyage du Vallat d

In [16]:
df_nbdechets.head()

,ID_RELEVE,type_regroupement,categorie,nb_dechet
0,44,GROUPE,ALLUMETTES,0.0
1,45,GROUPE,ALLUMETTES,0.0
2,46,GROUPE,ALLUMETTES,0.0
3,101,GROUPE,ALLUMETTES,0.0
4,102,GROUPE,ALLUMETTES,0.0


In [17]:
#Préparation des variables à conserver pour l'analyse 
cols_identifiers = ["ANNEE", "TYPE_MILIEU", "INSEE_COM", "DEP", "REG", "EPCI", "BV2022"]

#Colonnes de la base de données correspondant aux Volume global de chaque matériau
cols_volume = [k for k in df_enriched_bdd_zds.columns if "GLOBAL_VOLUME_" in k]

#Colonnes de la base de données correspondant au Nombre de déchets par regroupement (niveau 1-2-3-4)
#cols_nb_dechets = [k for k in df_enriched_bdd_zds.columns if "NB_DECHET_GROUPE_" in k]

#Colonnes de la base de données correspondant au Nombre de déchets par catégories DCSMM (uniquement niveau 4 et "déchets autre")
#cols_nb_dechets_DCSMM = [k for k in df_enriched_bdd_zds.columns if "NB_DECHET_DCSMM_" in k]

#Colonnes de la base de données correspondant au nombre de déchets par marque
#cols_nb_marque = [k for k in df_enriched_bdd_zds.columns if "NB_DECHET_MARQUE_" in k]

#Colonnes de la base de données correspondant au nombre de déchets par secteur
#cols_nb_secteur = [k for k in df_enriched_bdd_zds.columns if "NB_DECHET_SECTEUR_" in k]

#Colonnes de la base de données correspondant au nombre de déchets par filière REP
#cols_nb_REP = [k for k in df_enriched_bdd_zds.columns if "NB_DECHET_REP_" in k]


#### 0/ Vision d'ensemble des volumes catégorisés

In [18]:
#Vision d'ensemble
volume_total = df_enriched_bdd_zds["VOLUME_TOTAL"].sum()
poids_total = df_enriched_bdd_zds["POIDS_TOTAL"].sum()

#Volume total catégorisé
volume_total_categorise = df_enriched_bdd_zds[cols_volume].sum().sum()

volume_non_categorise = volume_total - volume_total_categorise

print("Vision d'ensemble des volumes présents dans la base : ")
print(f"- Volume total de déchets présents dans la base de données : {volume_total:.0f} Litres")
print(f"- Volume total de déchets catégorisés par matériau : {volume_total_categorise:.0f} Litres, soit {volume_total_categorise/volume_total:.0%} du volume total")
print(f"- Volume non catégorisé : {volume_non_categorise:.0f} litres soit {volume_non_categorise/volume_total:.0%} du volume total")

print(f"\nPoids total collecté : {poids_total:.0f} kg")



Vision d'ensemble des volumes présents dans la base : 
- Volume total de déchets présents dans la base de données : 3696524 Litres
- Volume total de déchets catégorisés par matériau : 2300935 Litres, soit 62% du volume total
- Volume non catégorisé : 1395588 litres soit 38% du volume total

Poids total collecté : 430971 kg


In [19]:
#Stats globales en nombre de déchets

#Nomre total catégorisé
print("\nVision d'ensemble des volumes présents dans la base : ")

for k in list(df_nbdechets["type_regroupement"].unique()):
    nb_dechets_regroupement = df_nbdechets[df_nbdechets["type_regroupement"] == k]["nb_dechet"].sum()
    print(f"- Nombre total de déchets catégorisés en fonction du {k} : {nb_dechets_regroupement:.0f} items")


Vision d'ensemble des volumes présents dans la base : 
- Nombre total de déchets catégorisés en fonction du GROUPE : 4060454 items
- Nombre total de déchets catégorisés en fonction du DCSMM : 788563 items
- Nombre total de déchets catégorisés en fonction du DCSMM_GENERIQUE : 11116 items
- Nombre total de déchets catégorisés en fonction du DCSMM_SPECIFIQUE : 25953 items
- Nombre total de déchets catégorisés en fonction du MARQUE : 84527 items
- Nombre total de déchets catégorisés en fonction du REP : 4080900 items
- Nombre total de déchets catégorisés en fonction du SECTEUR : 4080900 items


#### 1/ Typologie de matériau (volume)

In [21]:
#Dépivotage du tableau pour avoir une base de données exploitable
df_volume = df_enriched_bdd_zds.copy()
df_volume = df_volume.melt(id_vars = cols_identifiers, value_vars = cols_volume, var_name = "Type_matériau", value_name = "Volume_Litres")


#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_volume["Type_matériau"] = df_volume["Type_matériau"].str.replace("GLOBAL_VOLUME_","").str.title()
df_volume.head()

,ANNEE,TYPE_MILIEU,INSEE_COM,DEP,REG,EPCI,BV2022,Type_matériau,Volume_Litres
0,2019,Littoral (terrestre),13055,13,93,200054807,13055,Plastique,271.62
1,2019,Littoral (terrestre),13055,13,93,200054807,13055,Plastique,700.90
2,2019,Zone naturelle ou rurale (hors littoral et mon...,13022,13,93,200054807,13022,Plastique,420.00
3,2019,Cours d'eau,13055,13,93,200054807,13055,Plastique,1792.50
4,2019,Cours d'eau,34060,34,76,200022986,34111,Plastique,1560.00


In [22]:
#Répartition par type de matériau

#Grouper par type de matériau
df_group_mat = df_volume.groupby(["Type_matériau"])["Volume_Litres"].sum()
df_group_mat = df_group_mat.reset_index().sort_values(["Volume_Litres"], ascending = False)

#Afficher le graphique en camembert de la part de volume par matériau
fig = px.pie(df_group_mat, values='Volume_Litres', names='Type_matériau', title="Répartition du volume total collecté par type de matériaux")
fig.show()

In [23]:
#Grouper par année et type de matériau
df_group_mat_an = df_volume.groupby(["ANNEE", "Type_matériau"])["Volume_Litres"].sum()
df_group_mat_an = df_group_mat_an.reset_index().sort_values(["ANNEE", "Volume_Litres"], ascending = False)

#Graphique à barre empilées du volume total collecté par an et type de matériau
fig = px.bar(df_group_mat_an, x="ANNEE", y="Volume_Litres", color="Type_matériau",
             barmode = 'stack',
             title = "Volume collecté par an et par type de matériau",
             text_auto = True
             )
 
fig.show()

In [24]:
#Graphique à barre empilées du pourcentage de volume collecté par an et type de matériau
fig = px.histogram(df_group_mat_an, x="ANNEE", y="Volume_Litres", color="Type_matériau",
             barnorm = 'percent',
             title = "Part de volume collecté par an et par type de matériau",
             text_auto = True)
fig.update_layout(bargap=0.2)
fig.show()
print("Afficher uniquement à partir de 2020 ? (trop peu de données sur 2018 et 2019)")

Afficher uniquement à partir de 2020 ? (trop peu de données sur 2018 et 2019)


In [37]:
#Part de volume collecté par type de milieu

#Grouper par année et type de matériau
df_typemilieu = df_volume.groupby(["TYPE_MILIEU", "Type_matériau"] ,as_index = False)["Volume_Litres"].sum()
df_typemilieu = df_typemilieu.sort_values(["TYPE_MILIEU", "Volume_Litres"], ascending = False)

#Graphique à barre empilées du pourcentage de volume collecté par an et type de matériau
fig = px.histogram(df_typemilieu, x="TYPE_MILIEU", y="Volume_Litres", color="Type_matériau",
             barnorm = 'percent',
             title = "Répartition des matériaux en fonction du milieu",
             text_auto = False)
fig.update_layout(bargap=0.2)
fig.update_layout(yaxis_title="% du volume collecté", xaxis_title = None)
fig.show()

In [40]:
#Autre option : horizontal bar chart

#Graphique à barre empilées du volume total collecté par an et type de matériau
fig = px.bar(df_typemilieu, y="TYPE_MILIEU", x="Volume_Litres", color="Type_matériau",
             barmode = 'stack',
             title = "Volume collecté par matériau en fonction du milieu",
             text_auto = False
             )

#fig.update_layout(bargap=0.2)
#fig.update_layout(yaxis_title="% de déchets collectés")

fig.show()

#### 2/ Nombre de déchets par regroupement (groupes 1 à 4)

In [24]:
#Dépivotage du tableau pour avoir une base de données exploitable
df_nombre = df_enriched_bdd_zds.copy()
df_nombre = df_nombre.melt(id_vars = cols_identifiers, value_vars = cols_nb_dechets, var_name = "Type déchet", value_name = "Nombre de déchets") 
nb_total = df_nombre["Nombre de déchets"].sum()
print(f"Nombre total de déchets collectés dans la base (colonnes GROUPE) : {round(nb_total)}")

#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_nombre["Type déchet"] = df_nombre["Type déchet"].str.replace("NB_DECHET_GROUPE_","").str.title()
display(df_nombre.head())



Nombre total de déchets collectés dans la base (colonnes GROUPE) : 4039480


,ANNEE,TYPE_MILIEU,INSEE_COM,DEP,REG,EPCI,BV2022,Type déchet,Nombre de déchets
0,2019,Littoral (terrestre),13055,13,93,200054807,13055,Allumettes,NaN
1,2019,Littoral (terrestre),13055,13,93,200054807,13055,Allumettes,NaN
2,2019,Zone naturelle ou rurale (hors littoral et mon...,13022,13,93,200054807,13022,Allumettes,NaN
3,2019,Cours d'eau,13055,13,93,200054807,13055,Allumettes,NaN
4,2019,Cours d'eau,34060,34,76,200022986,34111,Allumettes,NaN


In [74]:
#Top 10 des déchets le plus représentés en nombre
df_type = df_nombre.groupby("Type déchet")["Nombre de déchets"].sum().sort_values(ascending = True).tail(10)

#Graphique à barres horizontales du nombre de déchets par type
fig = px.bar(df_type, x="Nombre de déchets", y=df_type.index,
             title = "Top 10 des déchets collectés en nombre",
             text_auto = True)
 
fig.show()

In [64]:
#Répartition des déchets par type et par an
df_type_an = df_nombre.groupby(["ANNEE","Type déchet"])["Nombre de déchets"].sum()
df_type_an = df_type_an.reset_index().sort_values(["ANNEE", "Nombre de déchets"], ascending = False)

#Graphique à barres empliées du nombre de déchets par type
fig = px.bar(df_type_an, x = "ANNEE", y="Nombre de déchets", 
             color="Type déchet",
             barmode = 'stack',
             title = "Part de déchets collectés en nombre")
 
fig.show()
print("Difficile à lire : il faudrait se limiter au TOP 10 des déchets les plus présents ?")

Difficile à lire : il faudrait se limiter au TOP 10 des déchets les plus présents ?


In [71]:
#Graphique à barre empilées du pourcentage de volume collecté par an et type de matériau
fig = px.histogram(df_type_an, x="ANNEE", y="Nombre de déchets", color="Type déchet",
             barnorm = 'percent',
             title = "Part de déchets collectés en nombre par an"
)
fig.update_layout(bargap=0.2)
fig.show()


#### 3/ Nombre de déchets par catégories DCSMM (uniquement niveau 4 et "déchets autre")

In [86]:
#Création d'une table dépivotée avec les types de dé&chets en colonne et le nombre en valeur 
df_dcsmm = df_enriched_bdd_zds.copy()
df_dcsmm = df_dcsmm.melt(id_vars = cols_identifiers, 
                         value_vars = cols_nb_dechets_DCSMM, 
                         var_name = "Type déchet DCSMM", 
                         value_name = "Nombre de déchets") 
nb_total = df_dcsmm["Nombre de déchets"].sum()
print(f"Nombre total de déchets collectés dans la base (colonnes DCSMM) : {nb_total:.0f}")

#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_dcsmm["Type déchet DCSMM"] = df_dcsmm["Type déchet DCSMM"].str.replace("NB_DECHET_DCSMM_","").str.title()
display(df_dcsmm.head())

Nombre total de déchets collectés dans la base (colonnes DCSMM) : 752689


,ANNEE,TYPE_MILIEU,INSEE_COM,DEP,REG,EPCI,BV2022,Type déchet DCSMM,Nombre de déchets
0,2019,Littoral (terrestre),13055,13,93,200054807,13055,Adhésifs,NaN
1,2019,Littoral (terrestre),13055,13,93,200054807,13055,Adhésifs,NaN
2,2019,Zone naturelle ou rurale (hors littoral et mon...,13022,13,93,200054807,13022,Adhésifs,NaN
3,2019,Cours d'eau,13055,13,93,200054807,13055,Adhésifs,NaN
4,2019,Cours d'eau,34060,34,76,200022986,34111,Adhésifs,NaN


In [88]:
#Top 10 des déchets le plus représentés en nombre
df_top10 = df_dcsmm.groupby("Type déchet DCSMM")["Nombre de déchets"].sum().sort_values(ascending = True).tail(10)

#Graphique à barres horizontales du nombre de déchets par type
fig = px.bar(df_top10, x="Nombre de déchets", y=df_top10.index,
             title = "Top 10 des déchets collectés en nombre (niveau de détail DCSMM)",
             text_auto = True)
 
fig.show()

In [89]:
#Répartition des déchets par type et par an
df_type_an = df_dcsmm.groupby(["ANNEE","Type déchet DCSMM"])["Nombre de déchets"].sum()
df_type_an = df_type_an.reset_index().sort_values(["ANNEE", "Nombre de déchets"], ascending = False)

#Graphique à barres empliées du nombre de déchets par type
fig = px.bar(df_type_an, x = "ANNEE", y="Nombre de déchets", 
             color="Type déchet DCSMM",
             barmode = 'stack',
             title = "Part de déchets collectés en nombre")
 
fig.show()

In [90]:
#Graphique à barre empilées du pourcentage de volume collecté par an et type de matériau
fig = px.histogram(df_type_an, x="ANNEE", y="Nombre de déchets", color="Type déchet DCSMM",
             barnorm = 'percent',
             title = "Part de déchets niveau DCSMM collectés en nombre par an"
)
fig.update_layout(bargap=0.2)
fig.show()


#### 4/ Nombre de déchets par secteur

In [93]:
#Création d'une table dépivotée avec les types de dé&chets en colonne et le nombre en valeur 
df_secteur = df_enriched_bdd_zds.copy()
df_secteur = df_secteur.melt(id_vars = cols_identifiers, 
                             value_vars = cols_nb_secteur, 
                             var_name = "Secteur", 
                             value_name = "Nombre de déchets") 

#Calcul du nombre total de déchets capturés dans les colonnes SECTEUR
nb_total = df_secteur["Nombre de déchets"].sum()
print(f"Nombre total de déchets collectés et catégorisés par secteur : {nb_total:.0f}")

#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_secteur["Secteur"] = df_secteur["Secteur"].str.replace("NB_DECHET_SECTEUR_","").str.title()
display(df_secteur.head())



Nombre total de déchets collectés et catégorisés par secteur : 3469495


,ANNEE,TYPE_MILIEU,INSEE_COM,DEP,REG,EPCI,BV2022,Secteur,Nombre de déchets
0,2019,Littoral (terrestre),13055,13,93,200054807,13055,Agriculture,0.0
1,2019,Littoral (terrestre),13055,13,93,200054807,13055,Agriculture,0.0
2,2019,Zone naturelle ou rurale (hors littoral et mon...,13022,13,93,200054807,13022,Agriculture,0.0
3,2019,Cours d'eau,13055,13,93,200054807,13055,Agriculture,0.0
4,2019,Cours d'eau,34060,34,76,200022986,34111,Agriculture,0.0


In [95]:
#Secteurs les plus représentés
df_top_secteur = df_secteur.groupby("Secteur")["Nombre de déchets"].sum().sort_values(ascending = True)

#Graphique à barres horizontales du nombre de déchets par type
fig = px.bar(df_top_secteur, x="Nombre de déchets", y=df_top_secteur.index,
             title = "Nombre de déchets collectés par secteur",
             text_auto = True)
 
fig.show()

In [98]:
#Top 10
df_top10secteur = df_top_secteur.tail(10)
fig = px.bar(df_top10secteur, x="Nombre de déchets", y=df_top10secteur.index,
             title = "Top 10 secteurs en nombre de déchets",
             text_auto = True)
 
fig.show()

#### 5/ Nombre de déchets par filière REP

In [100]:
#Création d'une table dépivotée avec les types de dé&chets en colonne et le nombre en valeur 
df_REP = df_enriched_bdd_zds.copy()
df_REP = df_REP.melt(id_vars = cols_identifiers, 
                             value_vars = cols_nb_REP, 
                             var_name = "Filiere_REP", 
                             value_name = "Nombre de déchets") 

#Calcul du nombre total de déchets capturés dans les colonnes SECTEUR
nb_total = df_secteur["Nombre de déchets"].sum()
print(f"Nombre total de déchets collectés et catégorisés par filière REP : {nb_total:.0f}")

#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_REP["Filiere_REP"] = df_REP["Filiere_REP"].str.replace("NB_DECHET_REP_","").str.title()


Nombre total de déchets collectés et catégorisés par filière REP : 3469495


In [101]:
#Filières REP les plus représentéës
df_group_REP = df_REP.groupby("Filiere_REP")["Nombre de déchets"].sum().sort_values(ascending = True)

#Graphique à barres horizontales du nombre de déchets par type
fig = px.bar(df_group_REP, x="Nombre de déchets", y=df_group_REP.index,
             title = "Nombre de déchets collectés par filière REP",
             text_auto = True)
 
fig.show()

#### 6/ Nombre de déchets par marque

In [104]:
#Création d'une table dépivotée avec les types de dé&chets en colonne et le nombre en valeur 
df_marques = df_enriched_bdd_zds.copy()
df_marques = df_marques.melt(id_vars = cols_identifiers, 
                             value_vars = cols_nb_marque, 
                             var_name = "Marque", 
                             value_name = "Nombre de déchets") 

#Calcul du nombre total de déchets capturés dans les colonnes SECTEUR
nb_total = df_marques["Nombre de déchets"].sum()
print(f"Nombre total de déchets collectés et catégorisés par marque : {nb_total:.0f}")

#Nettoyer le nom du Type déchet pour le rendre plus lisible
df_marques["Marque"] = df_marques["Marque"].str.replace("NB_DECHET_MARQUE_","").str.title()


Nombre total de déchets collectés et catégorisés par marque : 82011


In [105]:
#Secteurs les plus représentés
df_top_marques = df_marques.groupby("Marque")["Nombre de déchets"].sum().sort_values(ascending = True)

#Graphique à barres horizontales du nombre de déchets par type
fig = px.bar(df_top_marques, x="Nombre de déchets", y=df_top_marques.index,
             title = "Nombre de déchets collectés par marque",
             text_auto = True)
 
fig.show()

In [108]:
#Top 10
df_top10_marques = df_top_marques.tail(10)
fig = px.bar(df_top10_marques, x="Nombre de déchets", y=df_top10_marques.index,
             title = "Top 10 marques (identifiées) en nombre de déchets",
             text_auto = True)
 
fig.show()

#### Framents vs objets identifiables

In [55]:
df_DCSMM = df_nbdechets[df_nbdechets["type_regroupement"] == "DCSMM"]
categories_fragments = [k for k in df_DCSMM["categorie"].unique() if "FRAGMENT" in k]
df_DCSMM_fragments = df_DCSMM.loc[df_DCSMM["categorie"].isin(categories_fragments)]
df_DCSMM_objets = df_DCSMM.loc[~df_DCSMM["categorie"].isin(categories_fragments)]
total_fragments = df_DCSMM_fragments["nb_dechet"].sum()
total_objets = df_DCSMM_objets["nb_dechet"].sum()
print(f"Nb total de fragments : {total_fragments}")
print(f"Nb total d'objets identifiés: {total_objets}")

Nb total de fragments : 171175.0
Nb total d'objets identifiés: 617388.0


In [62]:
data ={
    "Type" : ["Fragments non identifiés", "Objets identifiés"],
     "Nombre de déchets" : [total_fragments, total_objets]
}

df_piechart = pd.DataFrame.from_dict(data)


# Création d'une échelle de couleurs bleues
colors = px.colors.sequential.Blues
colors[0] = '#1951A0'

# Création du diagramme en donut
fig = px.pie(df_piechart,
             values='Nombre de déchets',
             names='Type',
             title='Part des déchets identifiés et des fragments',
             hole=0.4,
             color='Nombre de déchets',
             color_discrete_sequence=colors)

# Amélioration de l'affichage
fig.update_traces(textinfo='percent+label')
fig.update_layout(width=800, height=600, legend_title_text='Type')

# Affichage du graphique
fig.show()

In [46]:
df_DCSMM.columns

Index(['ID_RELEVE', 'type_regroupement', 'categorie', 'nb_dechet'], dtype='object')